In [2]:
#libraries
import os
import geopandas as gpd
import rasterio
import numpy as np
import matplotlib.pyplot as plt

In [16]:
# Path to the folder containing the TIFF files
folder_path = "path_folder_GeoTIFF"

In [17]:
# Get a list of all TIFF files in the folder
tif_files = [f for f in os.listdir(folder_path) if f.endswith(".tif")]

In [18]:
# Function to process a TIFF file and return the hot and cold spots
def process_tiff(tif_path):
    with rasterio.open(tif_path) as src:
        # Read the raster data as a NumPy array
        temperature_matrix = src.read(1)  

        # Calculate quartiles for non-zero values
        non_zero_values = temperature_matrix[temperature_matrix != 0]
        q25 = np.percentile(non_zero_values, 25)
        q75 = np.percentile(non_zero_values, 75)

        # Find indices of cells with values above q75 and below q25
        hot_spot_indices = np.argwhere((temperature_matrix > q75) & (temperature_matrix != 0))
        cold_spot_indices = np.argwhere((temperature_matrix < q25) & (temperature_matrix != 0))
        
        return hot_spot_indices, cold_spot_indices

In [19]:
# Dictionary to store hot and cold spot arrays for each TIFF file
hot_spot_results = {}
cold_spot_results = {}

# Iterate through each TIFF file
for tif_file in tif_files:
    tif_path = os.path.join(folder_path, tif_file)
    
    # Process the TIFF file
    hot_spot_indices, cold_spot_indices = process_tiff(tif_path)
    
    # Store the hot and cold spot indices in the dictionaries
    hot_spot_results[tif_file] = hot_spot_indices
    cold_spot_results[tif_file] = cold_spot_indices


In [24]:
# Create a dictionary to count the occurrences of each cell position for hot and cold spots
hot_cell_count = {}
cold_cell_count = {}

# Iterate through hot and cold spot arrays from TIFF files
for tif_file, hot_spot_indices in hot_spot_results.items():
    for row, col in hot_spot_indices:
        cell_position = (row, col)
        
        # Increment the count for the cell position in hot spots
        if cell_position in hot_cell_count:
            hot_cell_count[cell_position] += 1
        else:
            hot_cell_count[cell_position] = 1

for tif_file, cold_spot_indices in cold_spot_results.items():
    for row, col in cold_spot_indices:
        cell_position = (row, col)
        
        # Increment the count for the cell position in cold spots
        if cell_position in cold_cell_count:
            cold_cell_count[cell_position] += 1
        else:
            cold_cell_count[cell_position] = 1

# Create datasets for hot and cold spots that occur more than 10 times
hot_spot_dataset = [(row, col, count) for (row, col), count in hot_cell_count.items() if count > 10] 
cold_spot_dataset = [(row, col, count) for (row, col), count in cold_cell_count.items() if count > 10]


In [25]:
len(cold_spot_dataset)

2707

In [26]:
len(hot_spot_dataset)

2657

In [39]:
# Load the original GeoTIFF you want to mark
original_tiff_path = "path_GeoTIFF"
with rasterio.open(original_tiff_path) as src:
    original_data = src.read(1)
    profile = src.profile

In [40]:
# Create arrays for marked hot spots and cold spots
hot_spot_marked = np.zeros_like(original_data)
cold_spot_marked = np.zeros_like(original_data)

In [41]:
# Define a value to mark hot spots (e.g., 255 for red)
hot_spot_value = 255

# Mark hot spots in the hot_spot_marked array
for row, col, _ in hot_spot_dataset:
    hot_spot_marked[row, col] = hot_spot_value


In [42]:
# Define a value to mark cold spots (e.g., -255 for blue)
cold_spot_value = -255

# Mark cold spots in the cold_spot_marked array
for row, col, _ in cold_spot_dataset:
    cold_spot_marked[row, col] = cold_spot_value

In [43]:
# Save marked hot spot and cold spot arrays as separate GeoTIFFs
output_directory = "output_folder" 
output_hot_spot_tiff = os.path.join(output_directory, "hot_spots_marked.tif")
with rasterio.open(output_hot_spot_tiff, 'w', **src.profile) as dst:
    dst.write(hot_spot_marked, 1)

In [44]:
output_cold_spot_tiff = os.path.join(output_directory, "cold_spots_marked.tif")
with rasterio.open(output_cold_spot_tiff, 'w', **src.profile) as dst:
    dst.write(cold_spot_marked, 1)